In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("C:/Users/bhudi/Desktop/New folder/git/sentiment bot/IMDB Dataset.csv")

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

In [5]:
X = data['review']
y = data['sentiment']

In [6]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [9]:
from sklearn.naive_bayes import MultinomialNB
sentiment_model = MultinomialNB()
sentiment_model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [10]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = sentiment_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8599333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.89      0.86      7474
           1       0.88      0.83      0.86      7526

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer
def chatbot(user_input):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(user_input)['compound']

    tfidf_input = tfidf_vectorizer.transform([user_input])
    predicted_sentiment = sentiment_model.predict(tfidf_input)[0]

    if sentiment_score >= 0.05:
        overall_sentiment = "positive"
    elif sentiment_score <= -0.05:
        overall_sentiment = "negative"
    else:
        overall_sentiment = "neutral"

    return f"Predicted sentiment: {predicted_sentiment}\nOverall sentiment: {overall_sentiment}"


In [13]:
user_input = input("Enter your review: ")
response = chatbot(user_input)
print(response)

Enter your review: wonderful movie
Predicted sentiment: 1
Overall sentiment: positive


In [12]:
import tkinter as tk
from tkinter import ttk, scrolledtext

def chatbot(user_input):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(user_input)['compound']

    tfidf_input = tfidf_vectorizer.transform([user_input])
    predicted_sentiment = sentiment_model.predict(tfidf_input)[0]

    if sentiment_score >= 0.05:
        overall_sentiment = "positive"
    elif sentiment_score <= -0.05:
        overall_sentiment = "negative"
    else:
        overall_sentiment = "neutral"

    return f"Predicted sentiment: {predicted_sentiment}\nOverall sentiment: {overall_sentiment}"

class SentimentAnalysisGUI:
    def __init__(self, master):
        self.master = master
        master.title("Sentiment Analysis Chatbot")

        self.label = tk.Label(master, text="Enter your review:")
        self.label.pack()

        self.user_input = tk.Entry(master, width=50)
        self.user_input.pack()

        self.button = tk.Button(master, text="Submit", command=self.submit, bg='green', fg='white')
        self.button.pack()

        self.output_text = scrolledtext.ScrolledText(master, width=50, height=10)
        self.output_text.pack()

    def submit(self):
        user_input = self.user_input.get()
        response = chatbot(user_input)
        self.output_text.insert(tk.END, f"\nUser: {user_input}\nBot: {response}\n")

if __name__ == "__main__":
    root = tk.Tk()
    gui = SentimentAnalysisGUI(root)
    root.mainloop()